#### Test mynds packages in the sandbox

In [ ]:
from pathlib import Path

from mynd.image import ImageType

from mynd.utils.filesystem import (
    list_directory,
    create_resource,
    Resource,
    ResourceManager,
)
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def main() -> None:
    """Main entrypoint."""

    DATADIR: Path = Path("/home/martin/data")

    image_directories: dict[str, Path] = {
        ImageType.COLOR: DATADIR
        / Path("acfr_images_preprocessed/r23685bc_20100605_021022_debayered"),
        ImageType.RANGE: DATADIR
        / Path("acfr_stereo_ranges/r23685bc_20100605_021022_ranges"),
        ImageType.NORMAL: DATADIR
        / Path("acfr_stereo_normals/r23685bc_20100605_021022_normals"),
    }

    IMAGE_PATTERN: str = "*.tiff"

    image_files: dict[ImageType, list[Path]] = {
        image_type: list_directory(directory, IMAGE_PATTERN)
        for image_type, directory in image_directories.items()
    }

    for image_type, files in image_files.items():
        logger.info(f"Image type: {image_type}, files: {len(files)}")

    # NOTE: Write your code here
    resource_manager: ResourceManager = ResourceManager()
    for image_type, files in image_files.items():
        resources: list[Resource] = [
            create_resource(path) for path in files if path.exists()
        ]

        resource_manager.add_resources(
            resources, tags=["image", str(image_type)]
        )

    query_results: dict[str, list[Resource]] = {
        "color": resource_manager.query_tags(["image", "color"]),
        "range": resource_manager.query_tags(["image", "range"]),
        "normal": resource_manager.query_tags(["image", "normal"]),
    }

    for category, resources in query_results.items():
        file_count: int = len(
            [resource for resource in resources if resource.is_file()]
        )
        dir_count: int = len(
            [resource for resource in resources if resource.is_directory()]
        )

        logger.info(f"Category: {category}, resources: {len(resources)}")
        logger.info(f" - Files:        {file_count}")
        logger.info(f" - Directories:  {dir_count}")


# INVOKE MAIN
main()